In [43]:
from ttsxai.utils import jupyter_utils
# jupyter_utils.register_always_reload(packages=[
#     'ttsxai.interfaces',
#     # 'ttsxai.models.text2mel',
#     'ttsxai.models.mel2wave',
#     'ttsxai.utils',
# ])
jupyter_utils.register_always_reload(modules=[
#     # 'ttsxai.text',
    'ttsxai.models.text2mel.tacotron2',
#     # 'ttsxai.models.text2mel.fastspeech2',
#     'ttsxai.utils.plot_utils',
#     'ttsxai.pitch_calculator',
#     'ttsxai.energy_calculator'
])

In [44]:
1

1

In [1]:
import os
import re
import argparse
import torch
import numpy as np
import IPython.display as ipd
import cloudpickle
from tqdm import tqdm
from scipy.io import wavfile

from torch.multiprocessing import Pool, set_start_method

from ttsxai.utils.utils import read_ljs_metadata
from ttsxai.interfaces.prosody_interface import ProsodyInterface
from ttsxai.interfaces.tts_interface import TTSInterface, get_text2mel, get_mel2wave

In [2]:
hop_length = 256
sampling_rate = 22050

In [3]:
text2mel = get_text2mel('tacotron2', 'cuda')
mel2wave = get_mel2wave('waveglow', 'cuda')

tts = TTSInterface(device='cuda', text2mel=text2mel, mel2wave=mel2wave ).to('cuda')

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



/nas/users/dahye/kw/tts/ttsxai/src/waveglow/glow.py:74: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2349.)
  W = torch.qr(torch.FloatTensor(c, c).normal_())[0]
/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/stft.py:67: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = pad_center(fft_window, filter_length)


### original case 분석 (have의 duration이 긴)

In [4]:
tts_dict = tts.forward('We have, therefore,')
text = tts_dict['text']
duration = tts_dict['duration']
phonesymbols = tts_dict['phonesymbols']
wave = tts_dict['wave']

print(text)
ipd.Audio(wave, rate=sampling_rate)

torch.Size([1, 512, 15])
torch.Size([1, 512, 15])
torch.Size([1, 512, 15])
encoder_outputs.shape= torch.Size([1, 15, 512])
(80, 127)
We have, therefore,


/nas/users/dahye/kw/tts/ttsxai/src/tacotron2/audio_processing.py:50: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  win_sq = librosa_util.pad_center(win_sq, n_fft)


In [5]:
for i in range(len(phonesymbols)):
    print(phonesymbols[i], duration[i])

W 4
IY1 13
  0
HH 10
AE1 23
V 12
, 0
  0
DH 4
EH1 5
R 7
F 13
AO2 12
R 13
, 11


In [6]:
words = []
for w in re.split(r"([,;.\-\?\!\s+])", text.replace('.', '')):
    if w.lower() not in ['.', ',', '”','“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        words.append(w.lower())

word_index_boundaries = []

s = 0 
for index, phone in enumerate(phonesymbols):
    if phone == " ":
        word_index_boundaries.append([s, e])
        s = index + 1
    elif phone not in ['.', ',', '”', '“', '"', "'", ';', '-', '(', ')', ':', '?', '!', ' ', '']:
        e = index
word_index_boundaries.append([s, e])

assert(len(words) == len(word_index_boundaries))

for idx, word_index_boundary in enumerate(word_index_boundaries):
    if sum(duration) == 1000:
        print('ignore sum(duration) == 1000')
        continue

    s, e = word_index_boundary
    # if words[idx] == 'have':
    print(words[idx], s, e, phonesymbols[s:e+1], np.sum(duration[s:e+1]))


we 0 1 ['W', 'IY1'] 17
have 3 5 ['HH', 'AE1', 'V'] 45
therefore 8 13 ['DH', 'EH1', 'R', 'F', 'AO2', 'R'] 54


### activation 수정코드 디버깅

In [7]:
pkl_file = '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ038-0144.pkl'
with open(pkl_file, "rb") as file:
    data = cloudpickle.load(file)

In [8]:
ref_text = data['text']
ref_duration = data['duration']
ref_phonesymbols = data['phonesymbols']
ref_wave = data['wave']
ref_activations = data['activations']
print(ref_text)
ipd.Audio(ref_wave, rate=sampling_rate)

that must have been some other time he picked me up, end quote,


In [9]:
tts.text2mel.set_modified_activations({})

tts_dict = tts.forward('We have, therefore,')
text = tts_dict['text']
duration = tts_dict['duration']
phonesymbols = tts_dict['phonesymbols']
wave = tts_dict['wave']
activations = tts_dict['activations']

print(text)

ipd.Audio(wave, rate=sampling_rate)

torch.Size([1, 512, 15])
torch.Size([1, 512, 15])
torch.Size([1, 512, 15])
encoder_outputs.shape= torch.Size([1, 15, 512])
(80, 132)
We have, therefore,


In [10]:
modified_activations = {
    'lstm': activations['lstm'].copy()
}
# modified_activations['lstm'][3:5+1, :] = ref_activations['lstm'][0:2+1, :].copy() # 긴 have -> that
modified_activations['lstm'][3:5+1, :] = ref_activations['lstm'][9:11+1, :].copy() # 긴 have -> 짧 have
# modified_activations['lstm'][3, :] = ref_activations['lstm'][9, :].copy() # 긴 have의 HH 바꾸기
# modified_activations['lstm'][4, :] = ref_activations['lstm'][10, :].copy() # 긴 have의 AE 바꾸기
# modified_activations['lstm'][5, :] = ref_activations['lstm'][11, :].copy() # 긴 have의 V 바꾸기
# modified_activations['lstm'][3:4+1, :] = ref_activations['lstm'][9:10+1, :].copy() # 긴 have의 HH, AE1 바꾸기
modified_activations['lstm'] = torch.from_numpy(modified_activations['lstm'][None, :]).cuda()

# modified_activations = {
#     'conv_2': activations['conv_2'].copy()
# }
# # modified_activations['conv_2'][3:5+1, :] = ref_activations['conv_2'][0:2+1, :].copy() # 긴 have -> that
# modified_activations['conv_2'][3:5+1, :] = ref_activations['conv_2'][9:11+1, :].copy() # 긴 have -> 짧 have
# # modified_activations['conv_2'][3, :] = ref_activations['conv_2'][9, :].copy() # 긴 have의 HH 바꾸기
# # modified_activations['conv_2'][3:4+1, :] = ref_activations['conv_2'][9:10+1, :].copy() # 긴 have의 HH, AE1 바꾸기
# modified_activations['conv_2'] = torch.from_numpy(modified_activations['conv_2'][None, :]).cuda()



tts.text2mel.set_modified_activations(modified_activations)
# tts.text2mel.set_modified_activations({})

tts_dict = tts.forward('We have, therefore,')
# text = tts_dict['text']
duration = tts_dict['duration']
phonesymbols = tts_dict['phonesymbols']
wave = tts_dict['wave']
# activations = tts_dict['activations']

for i in range(len(phonesymbols)):
    print(phonesymbols[i], duration[i])

print(text)
ipd.Audio(wave, rate=sampling_rate)

torch.Size([1, 512, 15])
torch.Size([1, 512, 15])
torch.Size([1, 512, 15])
### modify lstm
encoder_outputs.shape= torch.Size([1, 15, 512])
(80, 96)
W 5
IY1 12
  1
HH 5
AE1 5
V 7
, 0
  0
DH 5
EH1 6
R 7
F 12
AO2 8
R 11
, 12
We have, therefore,


### Happy의 HH, AE1로 Have 발음 수정하기

In [11]:
pkl_file = '/nas/users/dahye/kw/tts/ttsxai/data_activation/LJSpeech/tacotron2_waveglow/LJ012-0272.pkl'
with open(pkl_file, "rb") as file:
    data = cloudpickle.load(file)

ref_text2 = data['text']
ref_duration2 = data['duration']
ref_phonesymbols2 = data['phonesymbols']
ref_activations2 = data['activations']

In [12]:
modified_activations = {
    'lstm': activations['lstm'].copy()
}
# modified_activations['lstm'][3:5+1, :] = ref_activations['lstm'][0:2+1, :].copy() # 긴 have -> that
modified_activations['lstm'][3:4+1, :] = ref_activations2['lstm'][80:81+1, :].copy() # 긴 have -> 짧 have
# modified_activations['lstm'][3, :] = ref_activations['lstm'][9, :].copy() # 긴 have의 HH 바꾸기
# modified_activations['lstm'][4, :] = ref_activations['lstm'][10, :].copy() # 긴 have의 AE 바꾸기
# modified_activations['lstm'][5, :] = ref_activations['lstm'][11, :].copy() # 긴 have의 V 바꾸기
# modified_activations['lstm'][3:4+1, :] = ref_activations['lstm'][9:10+1, :].copy() # 긴 have의 HH, AE1 바꾸기
modified_activations['lstm'] = torch.from_numpy(modified_activations['lstm'][None, :]).cuda()


tts.text2mel.set_modified_activations(modified_activations)
# tts.text2mel.set_modified_activations({})

tts_dict = tts.forward('We have, therefore,')
# text = tts_dict['text']
duration = tts_dict['duration']
phonesymbols = tts_dict['phonesymbols']
wave = tts_dict['wave']
# activations = tts_dict['activations']

for i in range(len(phonesymbols)):
    print(phonesymbols[i], duration[i])

print(text)
ipd.Audio(wave, rate=sampling_rate)

torch.Size([1, 512, 15])
torch.Size([1, 512, 15])
torch.Size([1, 512, 15])
### modify lstm
encoder_outputs.shape= torch.Size([1, 15, 512])
(80, 120)
W 4
IY1 15
  0
HH 8
AE1 12
V 11
, 0
  0
DH 4
EH1 5
R 8
F 14
AO2 12
R 14
, 13
We have, therefore,


In [161]:
np.array(phonesymbols)

array(['W', 'IY1', ' ', 'HH', 'AE1', 'V', ',', ' ', 'DH', 'EH1', 'R', 'F',
       'AO2', 'R', ','], dtype='<U3')

In [93]:
ref_activations['lstm'].shape

(53, 512)

In [71]:
ref_phonesymbols

['DH',
 'AE1',
 'T',
 ' ',
 'M',
 'AH1',
 'S',
 'T',
 ' ',
 'HH',
 'AE1',
 'V',
 ' ',
 'B',
 'IH1',
 'N',
 ' ',
 'S',
 'AH1',
 'M',
 ' ',
 'AH1',
 'DH',
 'ER0',
 ' ',
 'T',
 'AY1',
 'M',
 ' ',
 'HH',
 'IY1',
 ' ',
 'P',
 'IH1',
 'K',
 'T',
 ' ',
 'M',
 'IY1',
 ' ',
 'AH1',
 'P',
 ',',
 ' ',
 'EH1',
 'N',
 'D',
 ' ',
 'K',
 'W',
 'OW1',
 'T',
 ',']